In [ ]:
from turtle import up
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch_geometric.data import Data
import networkx as nx
import matplotlib.pyplot as plt
import torch_geometric.utils as utils
from utils import *
from model import UpdateRule


height = 3
width = 2
hidden_dim = 16


n_inputs = 4
n_outputs = 4


# edges = build_edges(n_inputs, n_outputs, height, width)
# type_dict = {"hidden": [1, 0, 0], "input": [0, 1, 0], "output": [0, 0, 1]}
# total_hidden_dim = hidden_dim + len(type_dict["hidden"]) #hidden data + type
# n_nodes = height*width + n_inputs + n_outputs
# x = torch.zeros(n_nodes, total_hidden_dim)
# data = Data(edge_index=edges, x=x)
# graph = utils.to_networkx(data, to_undirected=True, remove_self_loops = True)
# nx.draw(graph)
# print(data.x.shape)

update_rule = UpdateRule(
    height,
    width,
    n_inputs, 
    n_outputs,
    hidden_dim,
    512
)
update_rule.draw()


In [ ]:
from datasets import *

# dataset_loader = DataLoader(TranslateDataset(), batch_size=1, shuffle=True)

meta_set = MetaDataset()

update_rule = UpdateRule(
    height,
    width,
    n_inputs, 
    n_outputs,
    hidden_dim,
    512
)
optimizer = torch.optim.Adam(update_rule.parameters(), lr=0.00001)

n_steps = 2
batch_size = 1
for epoch in range(1000000):
    for _ in range(batch_size):
        for set_idx in meta_set.iterate():
            update_rule.reset() 
            x = update_rule.initial
            
        
            for problem_data_x, problem_data_y in meta_set.get_set(set_idx):
                for i in range(n_steps):
                    output_node_data = torch.zeros(problem_data_y.shape)
                    x = update_rule(x, problem_data_x.float())
                    
                
            network_output = update_rule.get_output(x)
            loss = F.mse_loss(problem_data_y.float().squeeze(0), network_output)
            
            loss.backward()  
    optimizer.step()  
    optimizer.zero_grad()  
        
    print(f"\r Epoch {epoch * batch_size} | Loss {loss} | Network in: {problem_data_x}| Network out: {network_output.detach()} ", end="")
    # print(f"\r Epoch {epoch * batch_size} | Loss {loss}", end="")
    
    if epoch % (100 // batch_size) == 0:
        print()


In [ ]:
import numpy as np
np.set_printoptions(precision=3)

run_rule(torch.tensor([[0,0,0,1]]), update_rule)